### Importation 

In [45]:
import geemap
import ee 
import time

In [46]:
ee.Authenticate() # authentication 
ee.Initialize() # initialisation 

In [49]:
# Chargement

Map = geemap.Map()

senegal = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
roi = senegal.filter(ee.Filter.eq('country_na', "Ghana"))
Map.addLayer(roi,{}, "  Ghana ")
Map.centerObject(roi, 8)
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1').filterDate('2019-06-01', '2019-06-16').filterBounds(roi)

composite = ee.Algorithms.Landsat.simpleComposite(
    collection=landsat,
    asFloat=True
)

# Spécification des paramètres RGB pour l'affichage
rgb = {'bands': ['B4', 'B3', 'B2'], 'min': 0.0, 'max': 0.3}

# Ajout de la couche Landsat RGB à la carte, en la limitant à la région d'intérêt (roi)
Map.addLayer(composite.clip(roi), rgb, "Landsat RGB")

# Affichage de la carte
Map 





Map(center=[7.95944963018395, -1.2062283302187253], controls=(WidgetControl(options=['position', 'transparent_…

In [48]:
# Définir une région d'intérêt plus petite
roi_reduit = roi.geometry().bounds().buffer(distance=10000)  # Par exemple, une zone de 10 km autour de la région initiale

# Augmenter l'échelle
echelle_augmentee = 60  # Augmenter l'échelle pour réduire le nombre de pixels

# Exporter l'image vers Google Drive
tache_export = ee.batch.Export.image.toDrive(
    image=composite.clip(roi_reduit),  # Utiliser la composition Landsat et appliquer la ROI réduite
    description='Export Landsat Composite Senegal',  # Description de la tâche d'exportation
    folder="STAGE TOBLI/PROJECT",  # Dossier de destination dans Google Drive
    fileNamePrefix="test",  # Nom du fichier sans extension
    region=roi_reduit,  # Région d'intérêt pour l'exportation
    scale=echelle_augmentee,
      maxPixels=100113624 # Échelle en mètres par pixel
)
# Lancer la tâche d'exportation
tache_export.start()

# Attendre que la tâche d'exportation soit terminée
while tache_export.active():
    print('Exportation en cours...')
    time.sleep(5)

# Vérifier si la tâche d'exportation a réussi ou échoué
if tache_export.status()['state'] == 'COMPLETED':
    print('Exportation terminée avec succès.')
else: 
    print('Échec de l\'exportation :', tache_export.status())

Exportation en cours...
Exportation en cours...
Exportation en cours...
Exportation en cours...


KeyboardInterrupt: 